# LTE Survey

Ce programme traite les données géographiques du réseaux cellulaire 4G récoltées depuis un smartphone.
Révision 2024 - sur iPhone 11 Pro 

### Organisation des données

- Les données LTE seront extraites d'un fichier screencast du mode "Field Test" de l'iPhone
- Les données GPS se trouvent dans un fichiers .gpx, issus d'une application de traçage telle que Wikiloc

Les résulats:
- Ce processus retourne un fichier CSV contenant les points localisé de mesure du Numéro de Cellule 4G : CellId

### Remarques importantes

- Toutes les dates sont naïves, et convertie en heures locales

---

# Phase Initiale


1. Entrez le numéro de Survey (par exemple: 202)
2. Entrez le nom du dossier parent de ce Survey (par exemple: SURVEYS/Surveys_0200)
3. Entrez la distance minimum retenue entre deux relevés GPS (20 - 100) [mètres]

In [6]:
import sys
import os

# Get the directory of the notebook
notebook_dir = os.getcwd()

# Get the path to Folder corelte
corelte_path = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add corelte to Python path
sys.path.insert(0, corelte_path)

from corelte.argument import Argument, OCRMode

NETWORK_ID = 1     # 1=Swisscom; 4=Orange-F
SURVEY_ID = 313

#try:
argument = Argument(NETWORK_ID, SURVEY_ID)
#except Exception as e:
 #   raise Exception("\033[91m" + "Erreur: " + "\033[0m" + str(e))

argument.erase_png = False
argument.erase_txt = False
argument.OCRMode = OCRMode.MYOCR_PLUS
argument.save_to_db = True
argument.scale_factor = 0.5

OCR Mode: MYOCR_PLUS
Comment: coteau de Bernex - rampe de Chavant
Exclusions: [[0, 0]]
Survey Number: 313
Screencast file: /Volumes/HOME/kDrive/DATA/LTE/SURVEYS/Net_01/03/10/Survey_0313/Survey_01_0313.mp4
GPS file: /Volumes/HOME/kDrive/DATA/LTE/SURVEYS/Net_01/03/10/Survey_0313/Survey_01_0313.gpx


# Traite les données
Initialise le traitement

In [7]:
from corelte.fusion import Fusion
from corelte.helpers.helper import Helper
from corelte.screencast import Screencast
from corelte.track import Track
from corelte.orm.db import db_engines
from corelte.orm.models import Base

fusion = Fusion(argument)
helper = Helper()
screencast = Screencast(argument)
track = Track(argument)
    
#Base.metadata.drop_all(engine_test)
Base.metadata.create_all(db_engines.main)


2024-12-30 17:11:55,451 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-30 17:11:55,451 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-30 17:11:55,455 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-30 17:11:55,456 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-30 17:11:55,459 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-30 17:11:55,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-30 17:11:55,461 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-30 17:11:55,464 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

### Traite les données GPS

- Importe les coordonnées du GPS et les place dans une liste
- Etand la liste gps avec des points extrapolés toutes les 1 seconde 
- Calcule la vitesse et la direction de chaque point
- Sauve la liste du gps dans un fichier csv

In [8]:
track.read_gpx_file_into_lines_gps()
track.extend_gps_records_to_every_second()
track.calculate_speed_and_direction()
helper.save_csv_file(track.lines_gps, screencast.argument.tmp_gps_filename)

Successfully processed 303 GPS points


### Traite les données du Screencast

Extrait une image par seconde du screencast (si pas déjà fait)

In [9]:
screencast.split_video_into_frames()

Prépare la liste limitée des fichiers à passer à l'OCR

In [10]:
screencast.create_list_of_frames_to_ocr()

Extrait le texte de l'image à l'aide de MYOCR ou Tesseract (si pas déjà fait)

In [11]:
# lit toutes les valeurs écrites sur l'image (en deux passes)

if argument.OCRMode in [OCRMode.MYOCR, OCRMode.MYOCR_PLUS]:
    screencast.process_myocr_on_frames()

if argument.OCRMode == OCRMode.TESSERACT:
    screencast.process_tesseract_on_frames()


Importe les données ocr du screencast

In [12]:
""" screencast.process_video()
 """
if argument.OCRMode == OCRMode.MYOCR_PLUS:
    screencast.read_filtred_frames_files_into_linesMp4()
else:
    screencast.read_frame_files_into_linesMp4()

 i=3 words[0]=16:02 time_str=16:02
 i=57 words[0]=16:03 time_str=16:03


Ajuste le temps pour chaque mesure à la seconde près

In [13]:
screencast.set_precise_time_in_linesMp4_rows()

first_non_null_mp4_reading_time 2024-12-24 16:02:00
first_non_null_idx 3
first_minute_change_dt 2024-12-24 16:03:00
first_minute_change_idx 57
-----------------
first_non_null_mp4_reading_time 2024-12-24 16:02:06


Sauve la liste du screencast

In [14]:
helper.save_csv_file(screencast.linesMp4, screencast.argument.tmp_mp4_filename)

# Fusionne les données

In [15]:
fusion.fusion_data(track.cursor, track.lines_gps, screencast.cursor, screencast.linesMp4)

Nb de lignes fusionnées=820


Ote les exclusions

In [16]:
fusion.apply_exclusions()

Nb de lignes avant l'exclusion=820
Nb de lignes après les exclusions=820


Clarifie les points

Les points sont trop nombreux (1 par seconde). On filtre les données en fonction d'une distance minimale entre eux et du changement de cellule s'il y a lieu.

In [17]:
fusion.clarify_with_minimum_distance2()

Compense le retard de mesure dû à la vitesse

In [18]:
fusion.apply_speed_compensation_to_linesFusion()

# Sauve les données

In [19]:
helper.save_csv_file(fusion.linesFusion, fusion.argument.csv_filename)

if argument.save_to_db :
    fusion.save_linesFusion_to_database(track.cursor.first_non_null_reading_time)

DELETE FROM survey WHERE survey.network_id = :network_id_1 AND survey.survey_id = :survey_id_1
2024-12-30 17:12:29,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-30 17:12:29,388 INFO sqlalchemy.engine.Engine DELETE FROM survey WHERE survey.network_id = %(network_id_1)s AND survey.survey_id = %(survey_id_1)s
2024-12-30 17:12:29,388 INFO sqlalchemy.engine.Engine [generated in 0.00055s] {'network_id_1': 1, 'survey_id_1': 313}
2024-12-30 17:12:29,407 INFO sqlalchemy.engine.Engine SELECT cell.network_id, cell.cell_id, cell.tac, ST_AsEWKB(cell.geom) AS geom 
FROM cell 
WHERE cell.network_id = %(network_id_1)s AND cell.cell_id = %(cell_id_1)s
2024-12-30 17:12:29,407 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {'network_id_1': 1, 'cell_id_1': 17522437}
2024-12-30 17:12:29,428 INFO sqlalchemy.engine.Engine SELECT cell.network_id, cell.cell_id, cell.tac, ST_AsEWKB(cell.geom) AS geom 
FROM cell 
WHERE cell.network_id = %(network_id_1)s AND cell.cell_id = %(cell_id_1)s
2024-1

/Volumes/HOME/mext/CODE/LTE/corelte/fusion.py:166: SAWarning: relationship 'Reading_orm.survey' will copy column survey.network_id to column reading.network_id, which conflicts with relationship(s): 'Reading_orm.cell' (copies cell.network_id to reading.network_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="cell"' to the 'Reading_orm.survey' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  session.execute(stmt)


# Done